# Scenario
- A gas trading client wants to store natural gas during the summer and sell it in winter, expecting higher seasonal prices due to a colder-than-average forecast.


# My Task
- Build a prototype pricing model that calculates the net value of such storage contracts using forecasted natural gas prices and key cost inputs.

# Business Value
- This model helps the trading desk quickly estimate potential profit/loss from seasonal trades, guiding client negotiations and supporting automated quoting in future.

# Explanation
- The model forecasts future prices using Prophet, calculates buy/sell values for chosen injection/withdrawal dates, deducts storage and operational costs, and outputs the net contract values.

# Import the necessary libraries

In [1]:
import pandas as pd
from prophet import Prophet
from scipy.interpolate import interp1d
import numpy as np

c:\Users\user\Desktop\QuantitativeResearch\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### Process the dataset to be able to be fitted in prophet

In [3]:
# Load natural gas price data
df = pd.read_csv("Nat_Gas.csv")

In [8]:
# Convert the Dates to datetime
df['ds'] = pd.to_datetime(df['Dates'], format='%m/%d/%y')

- Prophet formatting
- ds	datestamp	datetime64[ns]
- y	the value to forecast	float or int
- Renames 'Prices' → 'y', because Prophet will throw an error if the column is not named 'y'.


In [9]:
df = df[['ds', 'Prices']].rename(columns={'Prices': 'y'})

- Fit the prophet model

In [10]:
model = Prophet()
model.fit(df)

18:22:03 - cmdstanpy - INFO - Chain [1] start processing
18:22:04 - cmdstanpy - INFO - Chain [1] done processing


- Create the forecasting for the next year based on my last data
- Forecast for 12 months

In [12]:
# Period 12 = months, M I want the in months
future = model.make_future_dataframe(periods=12, freq='ME')

- Make the prediction

In [13]:
forecast = model.predict(future)

In [15]:
forecast.head(5)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-10-31,10.181210,9.743990,10.102612,10.181210,10.181210,-0.251702,-0.251702,-0.251702,-0.251702,-0.251702,-0.251702,0.0,0.0,0.0,9.929508
1,2020-11-30,10.223802,10.294246,10.643241,10.223802,10.223802,0.256567,0.256567,0.256567,0.256567,0.256567,0.256567,0.0,0.0,0.0,10.480370
2,2020-12-31,10.267815,10.737399,11.084495,10.267815,10.267815,0.645077,0.645077,0.645077,0.645077,0.645077,0.645077,0.0,0.0,0.0,10.912891
3,2021-01-31,10.311827,10.785443,11.134789,10.311827,10.311827,0.639250,0.639250,0.639250,0.639250,0.639250,0.639250,0.0,0.0,0.0,10.951076
4,2021-02-28,10.351580,10.722169,11.079732,10.351580,10.351580,0.548535,0.548535,0.548535,0.548535,0.548535,0.548535,0.0,0.0,0.0,10.900115


### Exaplanation of Each Line
- ds: The date we're forecasting for.
- trend: The general direction the values are moving over time (upward or downward).
- yhat_lower: The lowest expected value (pessimistic guess).
- yhat_upper: The highest expected value (optimistic guess).
- trend_lower: The lowest the trend might realistically go.
- trend_upper: The highest the trend might realistically go.
- additive_terms: Effects like seasonality (e.g., holidays or monthly patterns) added to the trend.
- additive_terms_lower: The lowest those seasonal effects might be.
- additive_terms_upper: The highest those seasonal effects might be.
- yearly: The specific effect of repeating yearly patterns.
- yearly_lower: Minimum expected impact from the yearly pattern.
- yearly_upper: Maximum expected impact from the yearly pattern.
- multiplicative_terms: If seasonality changes depending on the value size (e.g., higher sales = bigger seasonal spike).
- multiplicative_terms_lower: Lowest impact of that change.
- multiplicative_terms_upper: Highest impact of that change.
- yhat: The predicted value — what the model thinks will happen.

- Forecast Dataframe
1. Pick the most important columns 'ds', 'yhat -> Forecasted Date and predicted value, rename them to be more easily understandable to non technical stakeholders
2. ds → Date: So it's clear this column holds the dates.
3. yhat → Price: So it's clear this column holds the predicted prices (or values you're forecasting).
4. This will make my data more readable and make me able to create charts

In [16]:
forecast_df = forecast[['ds', 'yhat']].rename(columns={'ds': 'Date', 'yhat': 'Price'})

In [17]:
forecast_df.set_index('Date', inplace=True)

# Create the pricing Function
- injection_dates -> A list of dates when natural gas is planned to be injected into storage.
- withdrawal_dates -> A list of dates when natural gas is planned to be withdrawn from storage.
- forecast_df -> A table (DataFrame) containing future gas price predictions over time.
- rate -> The maximum amount of gas you can inject or withdraw per day (or another time unit).
- max_volume -> The total storage capacity available (in cubic meters or MMBtu).
- storage_cost_per_month -> The monthly cost of storing gas ( rent for storage facility).
- contract_name -> The name or ID of the gas storage contract

## Business Value for each constraint
-  Use price forecasts to make profitable decisions.
- Respect real-world physical and financial limits.
- Plan smartly under a named contract.


## Linear Interpolation Function
- A function that estimates the gas price on any date by drawing straight lines between known forecast points.
- Because my injection - withdrawal dates might not exactly match the forecast dates so the interpolation gives a realistic price estimate for any in between day.

## Average buy price based on injection dates
- Mean of forecasted prices on all the days when gas is injected into storage.
- Why -> To find out how much my client is paying on average to purchase the gas you'll store.
- This helps compare against the future sell price and calculate your potential profit or loss.

## Average sell price based on withdrawal dates
-  Mean forecasted price on the days when gas is withdrawn from storage, which is when it's typically sold.
- Why -> To estimate the average revenue per unit of gas sold, helping you measure the profit margin when compared to the average buy price.


In [18]:

def evaluate_contract(
    injection_dates, withdrawal_dates, forecast_df,
    rate, max_volume, storage_cost_per_month,
    contract_name
):

    # Convert all injection and withdrawal dates to datetime format for accurate calculations
    injection_dates = [pd.to_datetime(d) for d in injection_dates]
    withdrawal_dates = [pd.to_datetime(d) for d in withdrawal_dates]

    # Extract the forecasted price series which is natural gas
    forecast_series = forecast_df['Price']

    # Create a linear interpolation function to estimate prices on any given date
    # This allows  to get price values even if the dates don't exactly match the forecast_df index
    interpolator = interp1d(
        forecast_series.index.astype(int),  # Convert dates to integer timestamps
        forecast_series.values, # Corresponding price values
        kind='linear', # Use straight-line approximation between points
        fill_value='extrapolate'  # Allow prediction beyond given date range
    )

    # Helper function : to get the estimated price for any specific date, use the interpolator (proposed by Chatgtp)
    def get_price(date):
        return float(interpolator(np.datetime64(date).astype(int)))

    # Calculate how much gas will be injected and withdrawn in total (based on rate and number of days)
    injected_volume = rate * len(injection_dates) # rate * how many days we are injecting days
    withdrawn_volume = rate * len(withdrawal_dates) # rate * how many days we are withdrawing

    # Check-up: How much gas is effectively traded , cannot exceed what was injected, withdrawn, or storage capacity.
    volume = min(injected_volume, withdrawn_volume, max_volume)

    # Compute average buy price based on injection dates, when gas is purchased
    buy_price = np.mean([get_price(d) for d in injection_dates])

    # Compute average sell price based on withdrawal dates, when gas is sold
    sell_price = np.mean([get_price(d) for d in withdrawal_dates])

    # Constraints given by JP Scenarion may differ to other problems

    # Gross profit: (sell price - buy price) × volume of gas
    gross_value = (sell_price - buy_price) * volume

    # Estimate how long the gas will be stored (in months) from first injection to last withdrawal
    storage_months = (withdrawal_dates[-1] - injection_dates[0]).days // 30

    # Total storage cost: monthly cost × number of months stored
    storage_cost = storage_months * storage_cost_per_month

    # Profit -> gross value minus the cost of storing the gas
    net_value = gross_value - storage_cost

    # Given by chatgtp to show my reasults better
    return {
        "📄 Contract": contract_name,
        "📆 Injection Dates": [str(d.date()) for d in injection_dates],
        "📆 Withdrawal Dates": [str(d.date()) for d in withdrawal_dates],
        "💰 Buy Price (USD/MMBtu)": round(buy_price, 2),
        "💵 Sell Price (USD/MMBtu)": round(sell_price, 2),
        "📦 Volume (MMBtu)": int(volume),
        "📈 Gross Value ($)": round(gross_value, 2),
        "🏢 Storage Cost ($)": round(storage_cost, 2),
        "✅ Net Contract Value ($)": round(net_value, 2)
    }


- Scenario 1

In [20]:
result = evaluate_contract(
    contract_name="Winter Hedge 2025-2026",
    injection_dates=['2025-07-31', '2025-08-31'],
    withdrawal_dates=['2025-12-31', '2026-01-31'],
    forecast_df=forecast_df,
    rate=1_000_000,
    max_volume=2_000_000,
    storage_cost_per_month=100_000
)

In [21]:
result

{'📄 Contract': 'Winter Hedge 2025-2026',
 '📆 Injection Dates': ['2025-07-31', '2025-08-31'],
 '📆 Withdrawal Dates': ['2025-12-31', '2026-01-31'],
 '💰 Buy Price (USD/MMBtu)': np.float64(-330.61),
 '💵 Sell Price (USD/MMBtu)': np.float64(-330.6),
 '📦 Volume (MMBtu)': 2000000,
 '📈 Gross Value ($)': np.float64(5618.79),
 '🏢 Storage Cost ($)': 600000,
 '✅ Net Contract Value ($)': np.float64(-594381.21)}

- loss of $594,381.21

In [23]:
# Scenario 1: Deep Winter Sale 2025
scenario_1 = evaluate_contract(
    contract_name="Deep Winter Sale 2025",
    injection_dates=['2025-06-30', '2025-07-31'],
    withdrawal_dates=['2025-12-31', '2026-01-31'],
    forecast_df=forecast_df,
    rate=1_000_000,
    max_volume=2_000_000,
    storage_cost_per_month=120_000
)

# Scenario 2: Autumn Arbitrage 2025
scenario_2 = evaluate_contract(
    contract_name="Autumn Arbitrage 2025",
    injection_dates=['2025-09-30', '2025-10-31'],
    withdrawal_dates=['2025-11-30', '2025-12-31'],
    forecast_df=forecast_df,
    rate=500_000,
    max_volume=1_000_000,
    storage_cost_per_month=80_000
)

# Display Results
print(scenario_1)
print(scenario_2)

{'📄 Contract': 'Deep Winter Sale 2025', '📆 Injection Dates': ['2025-06-30', '2025-07-31'], '📆 Withdrawal Dates': ['2025-12-31', '2026-01-31'], '💰 Buy Price (USD/MMBtu)': np.float64(-330.61), '💵 Sell Price (USD/MMBtu)': np.float64(-330.6), '📦 Volume (MMBtu)': 2000000, '📈 Gross Value ($)': np.float64(6757.24), '🏢 Storage Cost ($)': 840000, '✅ Net Contract Value ($)': np.float64(-833242.76)}
{'📄 Contract': 'Autumn Arbitrage 2025', '📆 Injection Dates': ['2025-09-30', '2025-10-31'], '📆 Withdrawal Dates': ['2025-11-30', '2025-12-31'], '💰 Buy Price (USD/MMBtu)': np.float64(-330.61), '💵 Sell Price (USD/MMBtu)': np.float64(-330.61), '📦 Volume (MMBtu)': 1000000, '📈 Gross Value ($)': np.float64(1120.09), '🏢 Storage Cost ($)': 240000, '✅ Net Contract Value ($)': np.float64(-238879.91)}


In [27]:
# Corrected Scenario 3: Peak Winter Play 2025
scenario_3 = evaluate_contract(
    contract_name="Peak Winter Play 2025",
    injection_dates=['2025-06-30', '2025-07-31'],
    withdrawal_dates=['2026-01-31', '2026-02-28'], 
    forecast_df=forecast_df,
    rate=1_000_000,
    max_volume=2_000_000,
    storage_cost_per_month=80_000
)


In [29]:
scenario_3

{'📄 Contract': 'Peak Winter Play 2025',
 '📆 Injection Dates': ['2025-06-30', '2025-07-31'],
 '📆 Withdrawal Dates': ['2026-01-31', '2026-02-28'],
 '💰 Buy Price (USD/MMBtu)': np.float64(-330.61),
 '💵 Sell Price (USD/MMBtu)': np.float64(-330.6),
 '📦 Volume (MMBtu)': 2000000,
 '📈 Gross Value ($)': np.float64(7840.6),
 '🏢 Storage Cost ($)': 640000,
 '✅ Net Contract Value ($)': np.float64(-632159.4)}

- Interpretability Function

In [34]:
def interpret_result(result):
    """
    Takes a dictionary returned by evaluate_storage_contract()
    and prints a summary of whether the contract was profitable or not.
    """
    contract_name = result.get("📄 Contract", "Unnamed Contract")
    net_value = result.get("✅ Net Contract Value ($)", 0)

    if net_value > 0:
        print(f" {contract_name}: PROFIT of ${net_value:,.2f}")
    elif net_value < 0:
        print(f" {contract_name}: LOSS of ${abs(net_value):,.2f}")
    else:
        print(f" {contract_name}: BREAK-EVEN (No profit or loss)")

In [35]:
print(interpret_result(scenario_1))
print(interpret_result(scenario_2))
print(interpret_result(scenario_3))

 Deep Winter Sale 2025: LOSS of $833,242.76
None
 Autumn Arbitrage 2025: LOSS of $238,879.91
None
 Peak Winter Play 2025: LOSS of $632,159.40
None
